In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imping
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder, MinMaxScaler


In [2]:
df = pd.read_csv("healthcare-dataset-stroke-data.csv")
df


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5105,18234,Female,80.0,1,0,Yes,Private,Urban,83.75,NaN,never smoked,0
5106,44873,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.0,never smoked,0
5107,19723,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.6,never smoked,0
5108,37544,Male,51.0,0,0,Yes,Private,Rural,166.29,25.6,formerly smoked,0


In [3]:
# Xóa cột 'id' vì không mang thông tin dự đoán
df = df.drop(columns=["id"])

# Điền giá trị thiếu trong cột 'bmi' bằng giá trị trung bình
df['bmi'] = df['bmi'].fillna(df['bmi'].mean())
df


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Yes,Private,Urban,228.69,36.600000,formerly smoked,1
1,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,28.893237,never smoked,1
2,Male,80.0,0,1,Yes,Private,Rural,105.92,32.500000,never smoked,1
3,Female,49.0,0,0,Yes,Private,Urban,171.23,34.400000,smokes,1
4,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.000000,never smoked,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Yes,Private,Urban,83.75,28.893237,never smoked,0
5106,Female,81.0,0,0,Yes,Self-employed,Urban,125.20,40.000000,never smoked,0
5107,Female,35.0,0,0,Yes,Self-employed,Rural,82.99,30.600000,never smoked,0
5108,Male,51.0,0,0,Yes,Private,Rural,166.29,25.600000,formerly smoked,0


In [4]:
# Danh sách các cột chứa dữ liệu phân loại (categorical)
# Các cột này có dạng chuỗi (text), ví dụ như 'Male', 'Female', 'Urban', 'Rural', v.v.
# Vì mô hình học máy chỉ hiểu số, nên chúng ta cần mã hóa các cột này thành số nguyên.
categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']

# Duyệt qua từng cột trong danh sách categorical_cols
for col in categorical_cols:
    le = LabelEncoder()  # Tạo một đối tượng LabelEncoder để mã hóa dữ liệu phân loại
    df[col] = le.fit_transform(df[col])  # Áp dụng mã hóa: gán mỗi giá trị văn bản thành một số nguyên

# Tách dữ liệu đầu vào (features) X từ DataFrame
# Chúng ta loại bỏ cột "stroke" vì đây là cột mục tiêu (label) cần dự đoán
X = df.drop(columns=["stroke"]).values  # X là tập hợp các đặc trưng đầu vào của mô hình

# Gán cột "stroke" làm nhãn đầu ra (target) y
# Đây là giá trị mà mô hình cần học để dự đoán (0: không đột quỵ, 1: có đột quỵ)
y = df["stroke"].values  # y là vector nhãn tương ứng với mỗi dòng trong X
df


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


In [5]:
# Tạo X (biến đầu vào) và y (nhãn đầu ra)
X = df.drop(columns=["stroke"]).values
y = df["stroke"].values
df


,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,1,67.0,0,1,1,2,1,228.69,36.600000,1,1
1,0,61.0,0,0,1,3,0,202.21,28.893237,2,1
2,1,80.0,0,1,1,2,0,105.92,32.500000,2,1
3,0,49.0,0,0,1,2,1,171.23,34.400000,3,1
4,0,79.0,1,0,1,3,0,174.12,24.000000,2,1
...,...,...,...,...,...,...,...,...,...,...,...
5105,0,80.0,1,0,1,2,1,83.75,28.893237,2,0
5106,0,81.0,0,0,1,3,1,125.20,40.000000,2,0
5107,0,35.0,0,0,1,3,0,82.99,30.600000,2,0
5108,1,51.0,0,0,1,2,0,166.29,25.600000,1,0


In [6]:
# Lấy lại dữ liệu X, y theo cách khác (có thể bỏ)
x = df.iloc[:, :10].values
y = df['stroke'].values
x, y


(array([[  1.        ,  67.        ,   0.        , ..., 228.69      ,
          36.6       ,   1.        ],
        [  0.        ,  61.        ,   0.        , ..., 202.21      ,
          28.89323691,   2.        ],
        [  1.        ,  80.        ,   0.        , ..., 105.92      ,
          32.5       ,   2.        ],
        ...,
        [  0.        ,  35.        ,   0.        , ...,  82.99      ,
          30.6       ,   2.        ],
        [  1.        ,  51.        ,   0.        , ..., 166.29      ,
          25.6       ,   1.        ],
        [  0.        ,  44.        ,   0.        , ...,  85.28      ,
          26.2       ,   0.        ]]),
 array([1, 1, 1, ..., 0, 0, 0], dtype=int64))

In [7]:
# Chuẩn hóa dữ liệu về khoảng [0, 1]
Scaler = MinMaxScaler()
X_new = Scaler.fit_transform(x)
X_new


array([[0.5       , 0.81689453, 0.        , ..., 0.80126489, 0.30126002,
        0.33333333],
       [0.        , 0.74365234, 0.        , ..., 0.67902317, 0.21298095,
        0.66666667],
       [0.5       , 0.97558594, 0.        , ..., 0.23451205, 0.25429553,
        0.66666667],
       ...,
       [0.        , 0.42626953, 0.        , ..., 0.12865848, 0.2325315 ,
        0.66666667],
       [0.5       , 0.62158203, 0.        , ..., 0.51320284, 0.17525773,
        0.33333333],
       [0.        , 0.53613281, 0.        , ..., 0.13922999, 0.18213058,
        0.        ]])

In [8]:
# Chia dữ liệu thành tập huấn luyện và kiểm tra (80% - 20%)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_new, y, test_size=0.2)


In [9]:
# Áp dụng mô hình Naive Bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, Y_train)


GaussianNB()

In [10]:
# Dự đoán kết quả trên tập kiểm tra
y_dudoan = model.predict(X_test)
y_dudoan, Y_test


(array([0, 0, 0, ..., 0, 0, 0], dtype=int64),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [11]:
# So sánh kết quả dự đoán và thực tế
y_compare = np.vstack((y_dudoan, Y_test))
print(y_compare)


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
# Thử dự đoán cho một bệnh nhân cụ thể
# gender = int(input("Giới tính (0: Nữ, 1: Nam): "))
# age = float(input("Tuổi: "))
# hypertension = int(input("Tăng huyết áp (0: Không, 1: Có): "))
# heart_disease = int(input("Bệnh tim (0: Không, 1: Có): "))
# ever_married = int(input("Đã kết hôn? (0: Chưa, 1: Rồi): "))
# work_type = int(input("Loại công việc (0: Trẻ em, 1: Chính phủ, 2: Riêng tư, 3: Tự kinh doanh, 4: Không rõ): "))
# Residence_type = int(input("Nơi sống (0: Nông thôn, 1: Thành thị): "))
# avg_glucose_level = float(input("Mức đường huyết trung bình: "))
# bmi = float(input("Chỉ số BMI: "))
# smoking_status = int(input("Hút thuốc (0: Chưa từng, 1: Thỉnh thoảng, 2: Hiện tại, 3: Không rõ): "))

# Đây là danh sách chứa các đặc trưng của một bệnh nhân:
# [giới tính, tuổi, đã kết hôn, loại công việc, nơi ở, hút thuốc, có bị bệnh tim, chỉ số glucose, chỉ số BMI, từng bị đột quỵ]
# Các giá trị này đã được mã hóa và chuẩn hóa trước đó giống như dữ liệu huấn luyện
dotquy = np.array([[0	,27.0,	0,	1,	1,	2,	1,	228.69,	25.600000,	1	]])

# Sử dụng mô hình đã huấn luyện để dự đoán xem bệnh nhân này có nguy cơ đột quỵ hay không
y_dotquy = model.predict(dotquy)

# Kiểm tra kết quả dự đoán và in ra thông điệp tương ứng
if y_dotquy[0] == 1:
    print("Bệnh nhân có nguy cơ bị đột quỵ.")  # Nếu kết quả dự đoán là 1, thông báo nguy cơ
else:
    print("Bệnh nhân không có dấu hiệu đột quỵ.")  # Nếu kết quả là 0, thông báo an toàn

# In ra nhãn dự đoán (dạng mảng, ví dụ [1] hoặc [0])
print(y_dotquy)


Bệnh nhân có nguy cơ bị đột quỵ.
[1]


In [13]:
# Tính độ chính xác của mô hình
accuracy = model.score(X_test, Y_test)
print("Accuracy:", accuracy)

# Nhân kết quả dự đoán với độ chính xác để tính mức độ nguy cơ
kq_nhan = y_dotquy[0] * accuracy

# Tạo DataFrame chứa mức độ nguy cơ
data = pd.DataFrame({
    'nguy co': [kq_nhan]
})
print(kq_nhan)


Accuracy: 0.8620352250489237
0.8620352250489237


In [14]:
# Hàm cảnh báo dựa trên giá trị nguy cơ đầu vào kq_nhan (kết quả nhận được)
def issue_warning(kq_nhan):
    # Nếu nguy cơ >= 0.8, cảnh báo mức RẤT CAO, yêu cầu kiểm tra y tế khẩn cấp
    if kq_nhan >= 0.8:
        return "Nguy cơ RẤT CAO - Yêu cầu kiểm tra y tế khẩn cấp!"
    # Nếu nguy cơ >= 0.6 và nhỏ hơn 0.8, cảnh báo mức CAO, nên khám định kỳ và thay đổi lối sống
    elif kq_nhan >= 0.6:
        return "Nguy cơ CAO - Nên khám định kỳ và thay đổi lối sống ngay."
    # Nếu nguy cơ >= 0.4 và nhỏ hơn 0.6, cảnh báo mức TRUNG BÌNH, theo dõi sức khỏe thường xuyên
    elif kq_nhan >= 0.4:
        return "Nguy cơ TRUNG BÌNH - Theo dõi sức khỏe thường xuyên."
    # Nếu nguy cơ nhỏ hơn 0.4, cảnh báo mức THẤP, duy trì lối sống lành mạnh
    else:
        return "Nguy cơ THẤP - Duy trì lối sống lành mạnh."

# Áp dụng hàm cảnh báo trên cột 'nguy co' của DataFrame 'data'
# Với mỗi giá trị trong cột 'nguy co', gọi hàm issue_warning và lưu kết quả vào cột mới 'cảnh_báo'
data['cảnh_báo'] = data['nguy co'].apply(issue_warning)

# In ra DataFrame 'data' sau khi đã thêm cột cảnh báo để kiểm tra kết quả
print(data)


    nguy co                                           cảnh_báo
0  0.862035  Nguy cơ RẤT CAO - Yêu cầu kiểm tra y tế khẩn cấp!


In [15]:
# Hàm đưa ra lời khuyên phòng ngừa dựa trên dữ liệu đầu vào dưới dạng mảng (array)
def prevention_advice_from_array(data):
    # Khởi tạo danh sách rỗng để chứa các lời khuyên
    advice = []

    # Giải nén (unpack) dữ liệu đầu vào: data[0] chứa một danh sách các giá trị thông tin sức khỏe
    gender, age, hypertension, heart_disease, ever_married, \
    work_type, residence_type, glucose, bmi, smoking_status = data[0]

    # Nếu có tiền sử tăng huyết áp (hypertension = 1), thêm lời khuyên kiểm soát huyết áp
    if hypertension == 1:
        advice.append("Kiểm soát huyết áp thường xuyên.")
    # Nếu có tiền sử bệnh tim mạch, thêm lời khuyên khám tim định kỳ
    if heart_disease == 1:
        advice.append("Khám tim mạch định kỳ.")
    # Nếu tình trạng hút thuốc là 'smokes' (mã 2), thêm lời khuyên ngừng hút thuốc
    if smoking_status == 2:  # 2 tương đương 'smokes'
        advice.append("Ngừng hút thuốc càng sớm càng tốt.")
    # Nếu loại hình công việc là 'Private' (mã 3), khuyên nghỉ ngơi, tránh stress
    if work_type == 3:  # 3 tương đương 'Private'
        advice.append("Nên nghỉ ngơi, tránh stress trong công việc.")
    # Nếu chỉ số BMI >= 30 (béo phì), khuyên giảm cân để giảm nguy cơ đột quỵ
    if bmi >= 30:
        advice.append("Giảm cân để giảm nguy cơ đột quỵ.")
    # Nếu mức glucose trong máu >= 140, khuyên kiểm tra đường huyết và điều chỉnh chế độ ăn
    if glucose >= 140:
        advice.append("Kiểm tra đường huyết, chế độ ăn phù hợp.")

    # Nếu có ít nhất một lời khuyên, nối các câu lại với dấu " | "
    # Nếu không có lời khuyên nào, trả về câu "Không có khuyến cáo đặc biệt."
    return " | ".join(advice) if advice else "Không có khuyến cáo đặc biệt."


In [16]:
# In kết quả dự đoán và lời khuyên
print(f"Dự đoán: {y_dotquy[0]} (0 = không đột quỵ, 1 = có đột quỵ)")
print(data)

result = prevention_advice_from_array(dotquy)
print("-----------------------------------------------------------")
print("Khuyến cáo phòng ngừa:", result)


Dự đoán: 1 (0 = không đột quỵ, 1 = có đột quỵ)
    nguy co                                           cảnh_báo
0  0.862035  Nguy cơ RẤT CAO - Yêu cầu kiểm tra y tế khẩn cấp!
-----------------------------------------------------------
Khuyến cáo phòng ngừa: Khám tim mạch định kỳ. | Kiểm tra đường huyết, chế độ ăn phù hợp.
